In [1]:
!pip install d3rlpy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [2]:
# =========================================
# 🚀 Offline Reinforcement Learning: Loan Approval
# =========================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from d3rlpy.algos import DiscreteCQLConfig
from d3rlpy.dataset import MDPDataset
from tensorflow.keras.models import load_model
import tensorflow as tf

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
# =========================================
# 1️⃣ Reward Function
# =========================================
def make_rewards(loan_amnt_arr, int_rate_arr, outcomes, actions):
    """
    Compute rewards:
      - If action == 0 (deny): reward = 0
      - If action == 1 and Fully Paid: +loan_amnt * int_rate
      - If action == 1 and Defaulted: -loan_amnt
    """
    int_frac = np.array(int_rate_arr, dtype=np.float32) / 100.0
    loan_amnt = np.array(loan_amnt_arr, dtype=np.float32)
    outcomes = np.array(outcomes, dtype=np.int32)
    actions = np.array(actions, dtype=np.int32)

    rewards = np.zeros_like(actions, dtype=np.float32)
    approve_paid = (actions == 1) & (outcomes == 0)
    approve_default = (actions == 1) & (outcomes == 1)

    rewards[approve_paid] = loan_amnt[approve_paid] * int_frac[approve_paid]
    rewards[approve_default] = -loan_amnt[approve_default]
    return rewards

In [6]:

# =========================================
# 2️⃣ Load Processed Data
# =========================================
processed_df = pd.read_csv("/content/drive/MyDrive/loan_data_processed.csv")

# Ensure numeric inputs only
num_cols = processed_df.select_dtypes(include=[np.number]).columns.tolist()
assert 'target' in num_cols, "❌ 'target' column missing in processed data!"
num_cols.remove('target')

X = processed_df[num_cols].values.astype('float32')
y = processed_df['target'].values.astype(int)

# Require unscaled versions for correct reward magnitudes
if 'loan_amnt' not in processed_df.columns or 'int_rate' not in processed_df.columns:
    raise ValueError("❌ loan_amnt and int_rate columns must exist for reward computation!")

loan_amnt = processed_df['loan_amnt'].values
int_rate = processed_df['int_rate'].values

# Split train/test
X_train, X_test, y_train, y_test, loan_train, loan_test, int_train, int_test = train_test_split(
    X, y, loan_amnt, int_rate, test_size=0.2, random_state=42, stratify=y
)

print(f"✅ Data split: Train {len(X_train)}, Test {len(X_test)}")


✅ Data split: Train 1078479, Test 269620


In [7]:
# =========================================
# 3️⃣ Build Offline Dataset (Augment Deny Actions)
# =========================================
actions_logged = np.ones((X_train.shape[0],), dtype=int)  # all historical approvals
rewards_logged = make_rewards(loan_train, int_train, y_train, actions_logged)
terminals = np.ones_like(rewards_logged, dtype=bool)

# Create augmented deny data (synthetic exploration)
obs_aug = np.concatenate([X_train, X_train], axis=0)
actions_aug = np.concatenate([actions_logged, np.zeros_like(actions_logged)], axis=0)
rewards_aug = np.concatenate([rewards_logged, np.zeros_like(rewards_logged)], axis=0)
terminals_aug = np.concatenate([terminals, np.ones_like(terminals)], axis=0)

# Normalize reward scale to stabilize training
reward_mean, reward_std = rewards_aug.mean(), rewards_aug.std() + 1e-8
rewards_norm = (rewards_aug - reward_mean) / reward_std

dataset = MDPDataset(
    observations=obs_aug.astype(np.float32),
    actions=actions_aug.astype(np.int32),
    rewards=rewards_norm.astype(np.float32),
    terminals=terminals_aug.astype(bool)
)

print(f"✅ Offline dataset ready: {dataset.transition_count} transitions")



2025-10-30 09:48.54 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float32')], shape=[(80,)]) reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)])
2025-10-30 09:48.54 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2025-10-30 09:49.02 [info     ] Action size has been automatically determined. action_size=2
✅ Offline dataset ready: 2156958 transitions


In [9]:
# =========================================
# 4️⃣ Configure and Train Offline RL (Discrete CQL)
# =========================================
from d3rlpy.models.encoders import DefaultEncoderFactory
from d3rlpy.models.q_functions import QRQFunctionFactory

config = DiscreteCQLConfig(
    learning_rate=1e-4,
    batch_size=512,
    encoder_factory=DefaultEncoderFactory(),
    q_func_factory=QRQFunctionFactory()   # Quantile regression for stability
    # alpha_threshold=10.0 # Removed as it's not a valid argument for DiscreteCQLConfig
)

cql = config.create(device="cuda" if tf.config.list_physical_devices('GPU') else "cpu")

# Estimate steps per epoch
steps_per_epoch = dataset.transition_count // 512
n_epochs = 30
n_steps = n_epochs * steps_per_epoch

print("🚀 Starting CQL training ...")
cql.fit(
    dataset,
    n_steps=n_steps,
    n_steps_per_epoch=steps_per_epoch,
    show_progress=True,
    # verbose=True # Removed as it's not a valid argument for QLearningAlgoBase.fit()
)
print("✅ CQL training completed.")

🚀 Starting CQL training ...
2025-10-30 09:53.13 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float32')], shape=[(80,)]), action_signature=Signature(dtype=[dtype('int32')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float32')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=2)
2025-10-30 09:53.13 [debug    ] Building models...            
2025-10-30 09:53.13 [debug    ] Models have been built.       
2025-10-30 09:53.13 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20251030095313
2025-10-30 09:53.13 [info     ] Parameters                     params={'observation_shape': [80], 'action_size': 2, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 512, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'compile_graph': False, 'learning_rate': 0.0001, 'op

Epoch 1/30:   0%|          | 0/4212 [00:00<?, ?it/s]

2025-10-30 09:54.18 [info     ] DiscreteCQL_20251030095313: epoch=1 step=4212 epoch=1 metrics={'time_sample_batch': 0.009505764541462956, 'time_algorithm_update': 0.005917527227320223, 'loss': 3.0844543419445793, 'td_loss': 2.3498225343702863, 'conservative_loss': 0.7346318037393312, 'time_step': 0.015521183652415914} step=4212
2025-10-30 09:54.18 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030095313/model_4212.d3


Epoch 2/30:   0%|          | 0/4212 [00:00<?, ?it/s]

2025-10-30 09:55.24 [info     ] DiscreteCQL_20251030095313: epoch=2 step=8424 epoch=2 metrics={'time_sample_batch': 0.009510648329826276, 'time_algorithm_update': 0.005906505179427854, 'loss': 2.9975397919940585, 'td_loss': 2.263198749631898, 'conservative_loss': 0.73434104173951, 'time_step': 0.015513129383750112} step=8424
2025-10-30 09:55.24 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030095313/model_8424.d3


...

2025-10-30 10:23.55 [info     ] DiscreteCQL_20251030095313: epoch=28 step=117936 epoch=28 metrics={'time_sample_batch': 0.00967027562522707, 'time_algorithm_update': 0.006027212453137549, 'loss': 2.9372969163618876, 'td_loss': 2.2081768614331088, 'conservative_loss': 0.7291200540655586, 'time_step': 0.015798817955643817} step=117936
2025-10-30 10:23.55 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030095313/model_117936.d3


Epoch 29/30:   0%|          | 0/4212 [00:00<?, ?it/s]

2025-10-30 10:25.02 [info     ] DiscreteCQL_20251030095313: epoch=29 step=122148 epoch=29 metrics={'time_sample_batch': 0.009598955469593363, 'time_algorithm_update': 0.005988405923671306, 'loss': 2.9347925457576287, 'td_loss': 2.2053945562665165, 'conservative_loss': 0.7293979893212984, 'time_step': 0.015690218689095262} step=122148
2025-10-30 10:25.02 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030095313/model_122148.d3


Epoch 30/30:   0%|          | 0/4212 [00:00<?, ?it/s]

2025-10-30 10:26.08 [info     ] DiscreteCQL_20251030095313: epoch=30 step=126360 epoch=30 metrics={'time_sample_batch': 0.009487925261853427, 'time_algorithm_update': 0.005953567546543799, 'loss': 2.938644832072661, 'td_loss': 2.209187679962102, 'conservative_loss': 0.7294571518416871, 'time_step': 0.015535646863812395} step=126360
2025-10-30 10:26.08 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20251030095313/model_126360.d3
✅ CQL training completed.


In [10]:
# =========================================
# 5️⃣ Evaluate RL Policy vs Baselines
# =========================================
policy_actions = cql.predict(X_test)
policy_rewards = make_rewards(loan_test, int_test, y_test, policy_actions)

avg_reward = policy_rewards.mean()
total_reward = policy_rewards.sum()

print(f"\n🎯 RL Policy Evaluation:")
print(f"Avg Reward per Loan: {avg_reward:.4f}")
print(f"Total Expected Reward: {total_reward:.2f}")
print(f"Approval Rate (RL policy): {policy_actions.mean():.2f}")

# Baselines
always_approve_rewards = make_rewards(loan_test, int_test, y_test, np.ones_like(policy_actions))
always_deny_rewards = make_rewards(loan_test, int_test, y_test, np.zeros_like(policy_actions))

print("\n🧩 Baseline Comparisons:")
print(f"Always-Approve Avg Reward: {always_approve_rewards.mean():.4f}")
print(f"Always-Deny Avg Reward: {always_deny_rewards.mean():.4f}")



🎯 RL Policy Evaluation:
Avg Reward per Loan: 0.0691
Total Expected Reward: 18621.95
Approval Rate (RL policy): 0.56

🧩 Baseline Comparisons:
Always-Approve Avg Reward: -0.0253
Always-Deny Avg Reward: 0.0000


In [11]:
# =========================================
# 6️⃣ Deep Learning Baseline Policy
# =========================================
try:
    dl_model = load_model("/content/drive/MyDrive/nn_best_model.h5")
    dl_probs = dl_model.predict(X_test, verbose=0).reshape(-1)

    # Approve if predicted default probability is below threshold
    threshold = 0.5
    dl_actions = (dl_probs < threshold).astype(int)

    dl_rewards = make_rewards(loan_test, int_test, y_test, dl_actions)

    print("\n🤖 Deep Learning Policy Baseline:")
    print(f"Threshold: {threshold}")
    print(f"DL Avg Reward: {dl_rewards.mean():.4f}")
    print(f"DL Total Reward: {dl_rewards.sum():.2f}")
    print(f"DL Approval Rate: {dl_actions.mean():.2f}")
except Exception as e:
    print("⚠️ Skipping DL baseline (model not found):", e)


🤖 Deep Learning Policy Baseline:
Threshold: 0.5
DL Avg Reward: 0.0098
DL Total Reward: 2652.09
DL Approval Rate: 0.58


In [12]:
# =========================================
# 7️⃣ Summary
# =========================================
print("\n📊 Summary of Policies:")
print(f"{'Policy':<25}{'Avg Reward':>15}{'Total Reward':>20}{'Approve %':>15}")
print(f"{'-'*75}")
print(f"{'RL (CQL) Policy':<25}{avg_reward:>15.4f}{total_reward:>20.2f}{policy_actions.mean():>15.2f}")
print(f"{'DL Baseline':<25}{dl_rewards.mean() if 'dl_rewards' in locals() else np.nan:>15.4f}"
      f"{dl_rewards.sum() if 'dl_rewards' in locals() else np.nan:>20.2f}"
      f"{dl_actions.mean() if 'dl_actions' in locals() else np.nan:>15.2f}")
print(f"{'Always Approve':<25}{always_approve_rewards.mean():>15.4f}{always_approve_rewards.sum():>20.2f}{1.00:>15.2f}")
print(f"{'Always Deny':<25}{always_deny_rewards.mean():>15.4f}{always_deny_rewards.sum():>20.2f}{0.00:>15.2f}")



📊 Summary of Policies:
Policy                        Avg Reward        Total Reward      Approve %
---------------------------------------------------------------------------
RL (CQL) Policy                   0.0691            18621.95           0.56
DL Baseline                       0.0098             2652.09           0.58
Always Approve                   -0.0253            -6834.06           1.00
Always Deny                       0.0000                0.00           0.00
